# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
!git clone --single-branch --branch 5-add-benchmark https://github.com/mozilla-ai/structured-qa

Cloning into 'structured-qa'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 823 (delta 169), reused 143 (delta 102), pack-reused 562 (from 1)
Receiving objects: 100% (823/823), 2.39 MiB | 5.13 MiB/s, done.
Resolving deltas: 100% (451/451), done.


In [2]:
%pip install ./structured-qa

Processing ./structured-qa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00
  Created wheel for structured-qa: filename=structured_qa-0.3.3.dev90+gd12fa72-py3-none-any.whl size=16325 sha256=2bc896037f77afbd304759ce187b8fcc29572a39f0f67cb4d0185c0

In [3]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00


# Setup

In [4]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [5]:
from loguru import logger

## Function to Process all questions for a single Section

In [6]:
ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You only answer based on the current information available.
The current information available is:

```
{CURRENT_INFO}
```

The answer must return ONLY one of the following strings and nothing else:
- YES/NO (for boolean questions)
Is the model an LLM?
YES
- Number (for numeric questions)
How many layers does the model have?
12
- Single letter (for multiple-choice questions)
What is the activation function used in the model? -A: ReLU -B: Sigmoid -C: Tanh
C
"""


def process_section_questions(
    section_file,
    section_data,
    model,
):
    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in section_data.iterrows():
        question = row["question"]
        logger.info(f"Question: {question}")
        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(
                    CURRENT_INFO=section_file.read_text()
                ),
            },
            {"role": "user", "content": question},
        ]
        response = model.get_response(messages)
        logger.info(f"Answer: {response}")
        answers[index] = response
        sections[index] = None
    return answers, sections

## Load Model

In [7]:
from structured_qa.model_loaders import load_llama_cpp_model

In [8]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2.5-7B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

# Run Benchmark

In [9]:
from pathlib import Path

import pandas as pd


logger.info("Loading input data")
data = pd.read_csv("structured-qa/benchmark/structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for section_name, section_data in data.groupby("section"):
    section_file = Path(f"structured-qa/benchmark/perfect_context/{section_name}.txt")

    answers, sections = process_section_questions(section_file, section_data, model)

    for index in section_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-01-30 12:40:07.774 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-01-30 12:40:07.811 | INFO     | __main__:process_section_questions:28 - Predicting
2025-01-30 12:40:07.815 | INFO     | __main__:process_section_questions:33 - Question: In billions, how many trainable parameters does GPT-3 have?
2025-01-30 12:40:09.567 | INFO     | __main__:process_section_questions:44 - Answer: 175
2025-01-30 12:40:09.570 | INFO     | __main__:process_section_questions:33 - Question: Does LoRA introduce additional inference latency compared to full fine-tuning?
2025-01-30 12:40:09.708 | INFO     | __main__:process_section_questions:44 - Answer: NO
2025-01-30 12:40:09.713 | INFO     | __main__:process_section_questions:28 - Predicting
2025-01-30 12:40:09.716 | INFO     | __main__:process_section_questions:33 - Question: What fire resistance must vertical partitions have? -A: EI30 -B: EI60 -C: EI90
2025-01-30 12:40:10.106 | INFO     | __main__:process_section_questions:44 - Answer:

# Results

In [12]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
10,10,https://arxiv.org/pdf/1706.03762,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,PDROP = 0.1,NaN
43,43,https://arxiv.org/pdf/2201.11903,3.4 Robustness of Chain of Thought,How many annotators provided independent chain...,3,2,NaN
47,47,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE EFFECTS,How many different races are there?,6,5,NaN
51,51,https://github.com/mozilla-ai/structured-qa/re...,CHAPTER OVERVIEW,"After taking a landmark tile, do you reveal a ...",NO,YES,NaN
52,52,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
65,65,https://github.com/mozilla-ai/structured-qa/re...,EXPEDITION PHASE,Do you need a fish to conquer a distant island?,YES,NO,NaN


In [13]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9393939393939394